# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [63]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/breannamitchell/Desktop/python-api-challenge/WeatherPy/output_data/cities_2.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chokurdakh,70.6333,147.9167,-20.24,95,100,2.54,RU,1680213519
1,1,yar-sale,66.8333,70.8333,-18.47,97,99,3.78,RU,1680213519
2,2,asyut,27.1810,31.1837,14.10,34,0,9.17,EG,1680213519
3,3,yellowknife,62.4560,-114.3525,-10.71,52,20,1.03,CA,1680213520
4,4,puerto ayora,-0.7393,-90.3518,30.98,88,67,3.58,EC,1680213520


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
%%capture --no-display

# Configure the map plot
humidity_bins = [0, 20, 40, 60, 80, 100]
humidity_labels = ['0-20', '21-40', '41-60', '61-80', '81-100']
city_data_df['Humidity Level'] = pd.cut(city_data_df['Humidity'], bins=humidity_bins, labels=humidity_labels)

map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',
    c='Humidity Level', 
    cmap='Category10',
    geo=True,
    hover_cols=['City', 'Country', 'Humidity'],
    alpha=0.7,
    tiles='CartoLight',
    background='#F0F0F0'
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity Level,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [65]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= 10) & (city_data_df['Max Temp'] <= 30) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Humidity Level
2,2,asyut,27.1810,31.1837,14.10,34,0,9.17,EG,1680213519,21-40
25,25,agadez,19.7500,10.2500,23.25,13,1,5.23,NE,1680213524,0-20
66,66,hami,42.8000,93.4500,11.25,22,42,3.48,CN,1680213530,21-40
88,88,khunti,23.0833,85.2833,20.06,38,96,2.17,IN,1680213534,21-40
96,96,pampierstad,-27.7832,24.6877,16.98,23,36,2.29,ZA,1680213535,21-40


### Step 3: Create a new DataFrame called `hotel_df`.

In [66]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,asyut,EG,27.1810,31.1837,34,
25,agadez,NE,19.7500,10.2500,13,
66,hami,CN,42.8000,93.4500,22,
88,khunti,IN,23.0833,85.2833,38,
96,pampierstad,ZA,-27.7832,24.6877,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000
params = { 
    "categories": "accommodation.hotel",
    "sort": "distance",
    "limit": 20,
    "apikey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
asyut - nearest hotel: No hotel found
agadez - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
khunti - nearest hotel: No hotel found
pampierstad - nearest hotel: No hotel found
avera - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
riyadh - nearest hotel: No hotel found
bolivar - nearest hotel: No hotel found
puerto del rosario - nearest hotel: No hotel found
ponta do sol - nearest hotel: No hotel found
fez - nearest hotel: No hotel found
forestville - nearest hotel: No hotel found
ayorou - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
shache - nearest hotel: No hotel found
najran - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found
dalby - nearest hotel: No hotel found
filingue - nearest hotel: No hotel found
korla - nearest hotel: No hotel found
nouadhibou - nearest hotel: No hotel found
blackwater - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
2,asyut,EG,27.1810,31.1837,34,No hotel found
25,agadez,NE,19.7500,10.2500,13,No hotel found
66,hami,CN,42.8000,93.4500,22,No hotel found
88,khunti,IN,23.0833,85.2833,38,No hotel found
96,pampierstad,ZA,-27.7832,24.6877,23,No hotel found
108,avera,US,33.1940,-82.5271,41,No hotel found
113,cabo san lucas,MX,22.8909,-109.9124,46,No hotel found
122,guerrero negro,MX,27.9769,-114.0611,46,No hotel found
128,riyadh,SA,24.6877,46.7219,23,No hotel found
150,bolivar,US,37.6145,-93.4105,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',
    c='Humidity Level', 
    cmap='Category10',
    geo=True,
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    alpha=0.7,
    tiles='CartoLight',
    background='#F0F0F0'
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)